In [1]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import os

Get the source code of the whole page for testing

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By


In [3]:
root = 'https://classroom.udacity.com/nanodegrees/nd101/parts/5b7b3a4c-10eb-4293-b821-ec6f539524dc/modules/df028333-45b5-4805-9d48-cb54a5ee1274/lessons/f680ebe0-9aa6-4183-ae4a-0cd30d07706b/project'

In [4]:
#setup option for chrome profile
chrome_options = Options()
# chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")

#start web driver
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
driver.implicitly_wait(60)
wait = WebDriverWait(driver, 30)

driver.get(root)

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


In [5]:
def sync_get_element_by_xpath(xpath):
    wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    return driver.find_element_by_xpath(xpath)

In [6]:
def sign_in():
    email = sync_get_element_by_xpath("//input[@data-testid='signin-email']")
    password = sync_get_element_by_xpath("//input[@data-testid='signin-password']")

    email.send_keys("huohsien@gmail.com")
    password.send_keys("jj1216")

    wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "button_primary__1qhjh.button__btn__2MHEg.form_primary-button__37eaW.button_standard__1p9sb")))
    sign_in_btn = driver.find_element_by_class_name("button_primary__1qhjh.button__btn__2MHEg.form_primary-button__37eaW.button_standard__1p9sb") 
    sign_in_btn.click()

In [7]:
sign_in()

In [8]:
def save_page_html():

    html = driver.page_source
    
    with open(file_name, 'w') as f:
        f.write(html)
#     print("html= ", html)
file_name = 'page_content.html'
save_page_html()

In [9]:
page_html = driver.page_source

In [10]:
page_soup = BeautifulSoup(page_html, 'html.parser')

In [11]:
style_elms = page_soup.find('head').find_all('style')

In [12]:
len(style_elms)

0

Start Post-processing the html file

In [13]:
file = '/Volumes/Pegasus32 R6/Udacity Deep Learning Nanodegree - Course Scraped/dlnd_course_downloaded/6. Deploying a Model/PROJECT - Project: Deploying a Sentiment Analysis Model/3. Project: Deploying a Sentiment Analysis Model/content.html'

In [14]:
def embed_tag(tag, html_str):
    return '<{}>'.format(tag) + html_str + '</{}>'.format(tag)

In [15]:
html_str = ''
lines = []
with open(file,'r') as fp:
    lines = fp.readlines()
html_str = ''.join(lines)

In [16]:
soup = BeautifulSoup(html_str, 'html.parser')

In [17]:
meta_elm = '<meta http-equiv="Content-type" content="text/html; charset=utf-8">'

In [18]:
def create_head_str():
    css_folder_path = 'css'
    css_filenames = [f for f in os.listdir(css_folder_path) if os.path.isfile(os.path.join(css_folder_path, f))]
    head_str = '<head>' + meta_elm
    for css_filename in css_filenames:
        link_str = '<link rel="stylesheet" href="../../../css/{}">'.format(css_filename)
        head_str = head_str + link_str
    for style_elm in style_elms:
        head_str = head_str + str(style_elm)
    
    head_str = head_str + '</head>'
    print("head_str: ", head_str)
    return head_str
head_str = create_head_str()

head_str:  <head><meta http-equiv="Content-type" content="text/html; charset=utf-8"><link rel="stylesheet" href="../../../css/0.63cd2.css"><link rel="stylesheet" href="../../../css/katex.min.css"><link rel="stylesheet" href="../../../css/app.7100f.css"><link rel="stylesheet" href="../../../css/13.9b4dc.css"><link rel="stylesheet" href="../../../css/1.99c19.css"><link rel="stylesheet" href="../../../css/2.a44a1.css"><link rel="stylesheet" href="../../../css/10.fe463.css"><link rel="stylesheet" href="../../../css/16.ca3a9.css"><link rel="stylesheet" href="../../../css/fonts.min.css"></head>


In [19]:
body_str = embed_tag('body', html_str)
content_str = head_str + body_str
html_str = embed_tag('html', content_str)

In [20]:
# update modification done in html string
soup = BeautifulSoup(html_str, 'html.parser')

In [21]:
div_to_be_remove_class_name = '_main--notification-bar--2AVbT'

In [22]:
div_to_be_removed_elms = soup.find_all('div',class_=div_to_be_remove_class_name)
for div_to_be_removed in div_to_be_removed_elms:
    div_to_be_removed.decompose()

In [23]:
# print(soup.prettify())

In [24]:
import requests
import os
from urllib.parse import urlparse

images_folder = 'img'

img_elms = soup.find_all('img')
for img_elm in img_elms:
    img_link = img_elm['src']
    r = requests.get(img_link, allow_redirects=True)
    a = urlparse(img_link)
    file_name = os.path.basename(a.path)                
    img_path = "./" + images_folder + "/" + file_name
    
    if not os.path.exists(images_folder):
        os.makedirs(images_folder)
    
    open(img_path, 'wb').write(r.content)
    print("file_name: ", file_name)
    # replace src in img element
    img_elm['src'] = "./img/{}".format(file_name)

In [25]:
path, name = os.path.split(file)
new_name = name[:-5] + '_processed.html'
new_file = os.path.join(path, new_name)
new_file

'/Volumes/Pegasus32 R6/Udacity Deep Learning Nanodegree - Course Scraped/dlnd_course_downloaded/6. Deploying a Model/PROJECT - Project: Deploying a Sentiment Analysis Model/3. Project: Deploying a Sentiment Analysis Model/content_processed.html'

In [26]:
with open(new_file, 'w', encoding='utf-8') as fp:
    fp.write(soup.prettify())